# Дектораторы

В этом домашнем задании мы напишем собственные дектораторы, которые будут менять системные объекты. Но для начала мы с вами познакомимся с функцией `write`.

In [81]:
import sys
from datetime import datetime
sys.stdout.write('Hello, my friend!')

Hello, my friend!

Это метод объектов file-like классов, то есть классов, которые реализуют семантику "Меня можно создать, из меня можно прочитать и в меня можно записать".

Самый главный пример такого объекта -- объект `file`, являющийся результатом вызова фукнции `open()`. Для простоты и универсальности взаимодействия, стандартный ввод и стандартный вывод тоже являются файлами, из которых можно читать и в которые можно писать. 

In [82]:
output = open("./some_test_data.txt", 'w')

In [83]:
output.write('123')

3

In [84]:
output.close()

Как вы могли заметить, функция возвращает число записанных байт. Это важная часть контракта, которую нужно поддержать, если вы хотите как-то подменять эту функцию.

## Задача 1

Для начала, давайте подменим метод `write` у объекта `sys.stdin` на такую функцию, которая перед каждым вызовом оригинальной функции записи данных в `stdout` допечатывает к тексту текущую метку времени.

In [85]:
original_write = sys.stdout.write

def my_write(string_text) -> str:
    """
    Выводит текущее дату и время + 'string_text'
    """
    time_string_text = f'{datetime.now().strftime("[%Y-%m-%d %H:%M:%S]:")} {string_text}'

    if string_text == '\n':
        original_write('\n')
    else:
        original_write(time_string_text)

sys.stdout.write = my_write

In [86]:
print('1, 2, 3')

[2021-12-14 13:24:54]: 1, 2, 3


In [87]:
sys.stdout.write = original_write

In [88]:
print('1, 2, 3')

1, 2, 3


Вывод должен был бы быть примерно таким:

```
[2021-12-05 12:00:00]: 1, 2, 3
```

## Задача 2

Упакуйте только что написанный код в декторатор. Весь вывод фукнции должен быть помечен временными метками так, как видно выше.

In [89]:
from typing import Callable

In [90]:
def timed_output(func: Callable) -> Callable:
    original_write = sys.stdout.write
    
    def wrapper(name: str) -> str:
        
        def my_write(string_text: str) -> str:
            time_string_text = f'{datetime.now().strftime("[%Y-%m-%d %H:%M:%S]:")} {string_text}'

            if string_text == '\n':
                original_write('\n')
            else:
                original_write(time_string_text)

        sys.stdout.write = my_write
        results = func(name)
        sys.stdout.write = original_write
        return results

    return wrapper

In [91]:
@timed_output
def print_greeting(name):
    print(f'Hello, {name}!')

In [92]:
print_greeting("Nikita")

[2021-12-14 13:24:57]: Hello, Nikita!


Вывод должен быть похож на следующий:

```
[2021-12-05 12:00:00]: Hello, Nikita!
```

## Задача 3

Напишите декторатор, который будет перенаправлять вывод фукнции в файл. 

Подсказка: вы можете заменить объект sys.stdout каким-нибудь другим объектом.

In [93]:
sys.stdout.write = original_write

In [94]:
def redirect_output(filepath: str) -> Callable:
    
    def outer_wrapper(func: Callable) -> Callable:
        
        def inner_wrapper():
            with open(filepath, 'w') as file:
                original_stdout = sys.stdout
                sys.stdout = file

                func()
                sys.stdout = original_stdout

        return inner_wrapper

    return outer_wrapper

In [95]:
@redirect_output('./function_output.txt')
def calculate():
    for power in range(1, 5):
        for num in range(1, 20):
            print(num ** power, end=' ')
        print()

In [96]:
calculate()

In [97]:
# %cat function_output.txt

In [98]:
with open('function_output.txt', 'r') as f1:
    print (f1.read())

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
1 4 9 16 25 36 49 64 81 100 121 144 169 196 225 256 289 324 361 
1 8 27 64 125 216 343 512 729 1000 1331 1728 2197 2744 3375 4096 4913 5832 6859 
1 16 81 256 625 1296 2401 4096 6561 10000 14641 20736 28561 38416 50625 65536 83521 104976 130321 

